In [ ]:
import json
import networkx as nx
from networkx.readwrite import json_graph

# Load JSON data for a single file
file_path = "amazon_test.json"  # Replace with your JSON file path
with open(file_path, "r") as file:
    json_data = json.load(file)

# Create a bipartite graph
B = nx.Graph()

# Process each user and their reviews
for user_data in json_data:
    user_id = user_data["id"]
    
    # Add user node
    B.add_node(user_id, bipartite=0, type="user")
    
    for review in user_data["profile"]:
        product_asin = review["productAsin"]
        
        # Add product node
        B.add_node(product_asin, bipartite=1, type="product")
        
        # Add edge with review details as attributes
        B.add_edge(
            user_id,
            product_asin,
            rating=review["rating"],
            title=review["title"],
            text=review["text"]
        )

# Debugging: Check graph nodes and edges
# print("Nodes:", B.nodes(data=True))
# print("Edges:", B.edges(data=True))

# Optional: Save the graph for later use
# nx.write_gpickle(B, "bipartite_graph.gpickle")

# Export graph to JSON schema
graph_data = json_graph.node_link_data(B)
with open("bipartite_graph_schema.json", "w") as json_file:
    json.dump(graph_data, json_file, indent=4)

In [ ]:
import os
import json
import networkx as nx
from networkx.readwrite import json_graph

#Load  a single file Directory containing JSON files
input_directory = "path/to/your/json_files"  # Replace with your input directory
output_directory = "path/to/your/output_graphs"  # Replace with your output directory

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate over all JSON files in the directory
for filename in os.listdir(input_directory):
    if filename.endswith(".json"):
        input_file_path = os.path.join(input_directory, filename)
        
        # Load JSON data
        with open(input_file_path, "r") as file:
            json_data = json.load(file)

        # Create a bipartite graph
        B = nx.Graph()

        # Process each user and their reviews
        for user_data in json_data:
            user_id = user_data["id"]
            
            # Add user node
            B.add_node(user_id, bipartite=0, type="user")
            
            for review in user_data["profile"]:
                product_asin = review["productAsin"]
                
                # Add product node
                B.add_node(product_asin, bipartite=1, type="product")
                
                # Add edge with review details as attributes
                B.add_edge(
                    user_id,
                    product_asin,
                    rating=review["rating"],
                    title=review["title"],
                    text=review["text"]
                )

        # Export graph to JSON schema
        graph_data = json_graph.node_link_data(B)

        # Save the graph JSON file with modified name
        output_file_name = f"GRAPH-{filename}"
        output_file_path = os.path.join(output_directory, output_file_name)
        with open(output_file_path, "w") as json_file:
            json.dump(graph_data, json_file, indent=4)

print("Graph JSON files have been created and saved in the output directory.")


In [ ]:
import plotly.graph_objects as go
from networkx.drawing.layout import spring_layout
import plotly.io as pio

pio.renderers.default = 'iframe'


# Sample the first 100 nodes and create a subgraph
sample_nodes = list(B.nodes())[:10]  # Get the first 100 nodes
sample_graph = B.subgraph(sample_nodes)  # Create a subgraph

# Generate layout for the sampled graph
pos = spring_layout(sample_graph)

# Extract node positions
x_nodes = [pos[node][0] for node in sample_graph.nodes()]
y_nodes = [pos[node][1] for node in sample_graph.nodes()]
node_labels = list(sample_graph.nodes())

# Extract edges
edge_x = []
edge_y = []
for edge in sample_graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Create plotly figure
fig = go.Figure()

# Add edges
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='gray'),
    hoverinfo='none',
    mode='lines'))

# Add nodes
fig.add_trace(go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers+text',
    text=node_labels,
    marker=dict(
        size=20,
        color=['lightblue' if sample_graph.nodes[node]['bipartite'] == 0 else 'lightgreen' for node in sample_graph.nodes()],
        line_width=2),
    textposition="top center"))

# Adjust layout
fig.update_layout(showlegend=False, hovermode='closest',
                  margin=dict(b=0, l=0, r=0, t=0))
fig.show()